In [1]:
from utilities.proj1_helpers import load_csv_data

In [ ]:
train_data = load_csv_data("data/train.csv")
y, x, ids = train_data[0], train_data[1], train_data[2]
N, D = x.shape
print(y.shape, x.shape, ids.shape)

(250000,) (250000, 30) (250000,)


In [ ]:
test_data = load_csv_data("data/test.csv")
y_train, x_train, ids_train = train_data[0], train_data[1], train_data[2]

In [ ]:
from regression.linear_models import *
from regression.stochastic_models import *
from utilities.cross_validation import cross_validation, build_k_indices, split_data
from regression.loss import compute_rmse_loss
from utilities.pca import compute_pca
from utilities.stochastic import *

In [ ]:
# Params to bruteforce
lambdas = np.logspace(-5, 1, 10)
degrees = np.arange(1, 8, 1)
k_folds = 4
seed = 1

In [ ]:
std = np.std(x, axis=0)
x /= np.ones((N, 1)).dot(std.reshape((1, D)))
x_reduced, vectors = compute_pca(x)

std = np.std(x_train, axis=0)
x_train /= np.ones((N, 1)).dot(std.reshape((1, D)))
g = np.mean(x_train, axis=0)
x_train -= g
x_train_reduced = x_train.dot(np.array(vectors).T) + np.ones((N, 1)).dot(g[:len(vectors)].reshape((1, len(vectors))))
indices = np.random.randint(0, high=len(x), size=((1, 500)))

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter3D(x_reduced[indices, 0], x_reduced[indices, 0], x_reduced[indices, 2])
plt.show()

In [ ]:
def generate_w(num_intervals):
    """Generate a grid of values for w0 and w1."""
    w0 = np.linspace(-100, 200, num_intervals)
    w1 = np.linspace(-100, 200, num_intervals)
    w2 = np.linspace(-100, 200, num_intervals)
    return w0, w1, w2


def get_best_parameters(w0, w1, w2, losses):
    """Get the best w from the result of grid search."""
    min_row, min_col, min_slice = np.unravel_index(np.argmin(losses), losses.shape)
    return losses[min_row, min_col, min_slice], w0[min_row], w1[min_col], w2[min_slice]

def grid_search(y, tx, w0, w1, w2):
    """Algorithm for grid search."""
    losses = np.zeros((len(w0), len(w1), len(w2)))
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO: compute loss for each combination of w0, w1 and w2.
    # ***************************************************
    for i in range(0, len(w0)):
        for j in range(0, len(w1)):
            for k in range(0, len(w2)):
                losses[i, j, k] = compute_rmse_loss(y, tx, np.array([w0[i], w1[j], w2[k]]))
    return losses

w0, w1, w2 =  generate_w(30)
losses = grid_search(y, x_reduced, w0, w1, w2)
print(get_best_parameters(w0, w1, w2, losses))

In [ ]:
x_reduced.shape

In [ ]:
best_loss = 1e100
best_params = (-1, -1)
k_indices = build_k_indices(y, k_folds, seed)
degree = 1

D = x_reduced.shape[1]
w0 = np.array([0 for i in range(D)])

for l in lambdas:
    w, loss_tr = stochastic_gradient_descent(y, x_reduced, w0, 1000, 1e-10, compute_rmse_loss, compute_mse_gradient)
    loss_te = compute_rmse_loss(y_train, x_train_reduced, w, lambda_=l)
    if loss_te < best_loss:
        best_loss = loss_te
        best_params = (l, degree)

    print("l={}, d={}: train {}, test {}".format(l, degree, loss_tr, loss_te))

In [ ]:
print(best_params, best_loss)

In [ ]:
def sums_(n_numbers, d):
    if n_numbers <= 1:
        result = d
    else:
        result = np.zeros((0, n_numbers))
        for i in range(d, -1, -1):
            rc = sums_(n_numbers - 1, d - i)
            print(rc, result.shape)
            result = np.vstack((result, np.hstack((i * rc, rc))))
    return result

X = x_reduced
n_obs, n_vars = X.shape
max_degree  = 4     # order of polynomial

stacked = np.zeros((0, n_vars)) #this will collect all the coefficients...    
for d in range(1, max_degree+1):
    stacked = np.vstack((stacked, sums_(n_vars, d)))

newX = np.zeros((X.shape[0], stacked.shape[0]))
for i in range(1, stacked.shape[0]+1):
    accumulator = np.ones((n_obs, 1))
    for j in range(1, n_vars+1):
        accumulator = accumulator * X[:, j-1]**stacked[i-1, j-1]
    newX[:, i-1] = accumulator

                                
                                
                